In [ ]:
'''import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2

import os

imgs = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pth = os.path.join(dirname, filename)
        if 'test' in filename or 'extra-image' in filename:
            imgs.append(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
df = pd.DataFrame(imgs,columns = ['path'])'''

In [ ]:
'''import cv2

df['shape'] = df.path.apply(lambda x:cv2.imread(x).shape)'''

In [ ]:
'''df['use'] = df['shape'].apply(lambda x:x[0]>=500 and x[1] >= 500)'''

In [ ]:
#df= df[df['use']]

In [ ]:
import pandas as pd
df = pd.read_csv('../input/cassava-leaf-disease-merged/merged.csv')
df['path'] = '../input/cassava-leaf-disease-merged/train/'+df['image_id']

In [ ]:
df.head()

In [ ]:
from PIL import Image
from albumentations.core.composition import Compose
from albumentations.pytorch import ToTensorV2
from collections import defaultdict, deque
# from efficientnet_pytorch import EfficientNet
from pytorch_lightning import Callback
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from torch import nn
from torch.autograd import Variable




from torch.utils.data import Dataset, TensorDataset, DataLoader
from torchvision.transforms import functional as F
from typing import Any, Dict, List, Union, Optional
import albumentations as A
import ast
import collections
import copy
import cv2
import datetime
import importlib
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
# import pretrainedmodels
import pytorch_lightning as pl
%matplotlib inline
import random
import seaborn as sns
import shutil
import tempfile
import time
import torch
import torch.distributed as dist
import torch.nn.functional as F
import torch.utils.data
import pandas as pd
import torchvision
from tqdm.notebook import tqdm
sns.set_style('darkgrid')
import albumentations as A
from albumentations.pytorch import ToTensorV2
path = '/kaggle/input/cassava-leaf-disease-classification/'

In [ ]:
hparams ={}
hparams['sizes'] = [384]
hparams['path'] = ['../input/one-more-time/epoch20-metric0.8932.ckpt'
                   ]
hparams['aug'] = A.Compose([

    A.CenterCrop(500,500),
    A.Resize(height=hparams['sizes'][0], width=hparams['sizes'][0], p=1.0),
    A.Normalize(),
    ToTensorV2(),
])

#'../input/leafclassification-modificated-size-cfe02b/model/last.ckpt'

In [ ]:
class ImageClassificationDataset(Dataset):
    def __init__(
        self,
        image_names,
        aug: Compose,
        labels: Optional[List[int]],
        img_path: str = '',
        mode: str = 'train',
        labels_to_ohe: bool = False,
        n_classes: int = 5,
    ):
        """
        Image classification dataset.

        Args:
            df: dataframe with image id and bboxes
            mode: train/val/test
            img_path: path to images
            transforms: albumentations
        """
        self.df = image_names
        self.mode = mode
        self.aug = aug
        self.img_path = img_path
        self.image_names = image_names
        if labels is not None:
            if not labels_to_ohe:
                self.labels = np.array(labels)
            else:
                self.labels = np.zeros((len(labels), n_classes))
                self.labels[np.arange(len(labels)), np.array(labels)] = 1

    def __getitem__(self, idx: int) -> Dict[str, np.array]:
        image_path = self.df.loc[idx]['path']#self.img_path + self.image_names[idx]
        '''image = cv2.imread(f'{image_path}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)'''
        image = cv2.imread(f'{image_path}')[:,:,::-1]
        if image is None:
            raise FileNotFoundError(image_path)
        target = self.labels[idx]

        img = self.aug(image=image)['image']
        sample = {'image_path': image_path, 'image': img, 'target': np.array(target).astype('int64'),'shape':(image.shape[0],image.shape[1])}

        return sample

    def __len__(self) -> int:
        return len(self.image_names)

In [ ]:
import torchvision
class ModelCassava(nn.Module):

    def __init__(self):
        super().__init__()

        model = torchvision.models.resnext50_32x4d(pretrained=False)#.children()))[:-1]
       
        self.start = nn.Sequential(*[model.conv1,model.bn1,model.relu,model.maxpool])
        self.layer1 = model.layer1
        self.layer2 = model.layer2
        self.layer3 = model.layer3
        self.layer4 = model.layer4
        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        
        self.out1 = nn.Linear(1024,5)
        self.out2 = nn.Linear(2048,5)
    def forward(self,image):
        
        out0 = self.layer3(self.layer2(self.layer1(self.start(image))))
        out1 = self.out1(self.avg_pool(out0).squeeze(2).squeeze(2))
        out2 = self.out2(self.avg_pool(self.layer4(out0)).squeeze(2).squeeze(2))
        
        #loss = (self.config.loss(out1, y) + self.config.loss(out2, y))/2
        
        y_hat = (out1.softmax(axis=1) + out2.softmax(axis=1))/2
        return y_hat
   
import torch
import pytorch_lightning as pl

class Model(pl.LightningModule):
    def __init__(self,model):
        super().__init__()
        self.loss = F.cross_entropy#LabelSmoothSoftmaxCEV1()
        self.model = model
    def forward(self, x):
        return self.model(x)

In [ ]:
df.drop([21419, 21697, 21743, 21932, 21982, 22036, 22125, 22308, 22832,
            22850, 23219, 24104, 24496, 24592, 24897, 24959, 25080, 25270,
            25319, 25510, 25771, 25882, 25898, 25964, 25988, 26274, 26301],inplace=True)
df = df.reset_index(drop=True)
sub =df

sub.head()

In [ ]:

valid_augs = hparams['aug']
test_dataset = ImageClassificationDataset(image_names=df,
                                                        aug=valid_augs,
                                                        labels=df['label'].values,
                                                        img_path='',
                                                        mode='test',
                                                        labels_to_ohe=False,
                                                        n_classes=5)



In [ ]:
lit_model = Model(ModelCassava())   

lit_model.load_state_dict(torch.load(hparams['path'][0])['state_dict'])
lit_model.net = lit_model.model.eval().to(torch.device('cuda'))

In [ ]:

def get_spm(input,target,conf,model):

    imgsize = (conf.cropsize,conf.cropsize)
    bs = input.size(0)
    with torch.no_grad():
        output,fms,_ = model(input)
        if 'inception' in conf.netname:
            clsw = model.module.fc
        else:
            clsw = model.module.classifier
        weight = clsw.weight.data
        bias = clsw.bias.data
        weight = weight.view(weight.size(0),weight.size(1),1,1)
        fms = F.relu(fms)
        poolfea = F.adaptive_avg_pool2d(fms,(1,1)).squeeze()
        clslogit = F.softmax(clsw.forward(poolfea))
        logitlist = []
        for i in range(bs):
            logitlist.append(clslogit[i,target[i]])
        clslogit = torch.stack(logitlist)

        out = F.conv2d(fms, weight, bias=bias)

        outmaps = []
        for i in range(bs):
            evimap = out[i,target[i]]
            outmaps.append(evimap)

        outmaps = torch.stack(outmaps)
        if imgsize is not None:
            outmaps = outmaps.view(outmaps.size(0),1,outmaps.size(1),outmaps.size(2))
            outmaps = F.interpolate(outmaps,imgsize,mode='bilinear',align_corners=False)

        outmaps = outmaps.squeeze()

        for i in range(bs):
            outmaps[i] -= outmaps[i].min()
            outmaps[i] /= outmaps[i].sum()


    return outmaps,clslogit

In [ ]:
def extract_mask(model, img, shape):
    res=model.net.layer4(lit_model.net.layer3(lit_model.net.layer2(lit_model.net.layer1(lit_model.net.start(img.to(torch.device('cuda'))))))).sum(axis=1)[0]
    res=F.interpolate(res.unsqueeze(0).unsqueeze(0),size=shape,mode='bilinear',align_corners=False)

    res = res-res.min()
    #res = res/res.sum()
    return res[0][0]

In [ ]:
res=extract_mask(lit_model, test_dataset[7]['image'].unsqueeze(0), (500,500))

In [ ]:
res=res.cpu().detach().numpy()


In [ ]:
plt.imshow(res)#>700)*255)

In [ ]:
plt.imshow(test_dataset[7]['image'].permute(1,2,0))

In [ ]:
#plt.hist(res.cpu().detach().numpy().flatten())

In [ ]:
def crop(size1,size2):
    width, height = size1   # Get dimensions
    new_width, new_height = size2
    left = (width - new_width)/2
    top = (height - new_height)/2
    right = (width + new_width)/2
    bottom = (height + new_height)/2
    return int(left),int(top),int(right),int(bottom)

In [ ]:
test_dataset[0]['image_path'].split('/')[-1]

In [ ]:
os.mkdir('masks')

In [ ]:
os.mkdir('masks_zip')

In [ ]:
from tqdm import tqdm_notebook
i=0
for x in tqdm_notebook(test_dataset):
    mask = extract_mask(lit_model, x['image'].unsqueeze(0), (500,500)).cpu().detach().numpy()
    left,top,right,bottom = crop(x['shape'],(500,500))
    save_img = np.zeros(x['shape'])
    save_img[left:right,top:bottom] = mask
    cv2.imwrite('masks/'+x['image_path'].split('/')[-1],(255*(save_img/save_img.max())).astype(np.uint8))    
    '''if i ==2:
        break
    i+=1'''

In [ ]:
plt.imshow(save_img)

In [ ]:
os.listdir('masks')[:7]

In [ ]:
import zipfile

def zip_and_remove(path):
    ziph = zipfile.ZipFile(f'{path}.zip', 'w', zipfile.ZIP_DEFLATED)
    
    for file in tqdm(os.listdir('masks')):
            file_path = os.path.join('masks', file)
            ziph.write(file_path)
            os.remove(file_path)
    
    ziph.close()

In [ ]:
zip_and_remove('masks_zip')

In [ ]:
'''def test_pred(hparams,test_dataset):
    res = 0

    for path in hparams['path']:
        lit_model = Model(ModelCassava())   

        lit_model.load_state_dict(torch.load(path)['state_dict'])
        lit_model.net = lit_model.model.eval().to(torch.device('cuda'))
        print('==========================')
        for size in hparams['sizes']:
            res_size = []
            test_dataset.aug =  A.Compose([ A.CenterCrop(500,500),A.Resize(height=size, width=size, p=1.0),A.Normalize(),ToTensorV2()])
            for batch in  torch.utils.data.DataLoader(
                test_dataset,
                batch_size=2,
                num_workers=4,
                shuffle=False,
            ):

                image = batch['image'].to('cuda')
                target = batch['target'].to('cuda')
                with torch.no_grad():
                    outputs = lit_model.net(image)
                    preds = outputs.softmax(axis=1).detach().cpu()

                    res_size.append(preds)
            res = res + torch.cat(res_size)
    return res'''

In [ ]:
#predictions = test_pred(hparams,test_dataset)

In [ ]:
'''predictions = []

for batch in test_loader:

    image = batch['image'].to('cuda')
    target = batch['target'].to('cuda')
    with torch.no_grad():
        outputs = lit_model.net(image)
        preds = outputs.argmax(1).detach().cpu().numpy()

        predictions.append(preds)'''
#sub['label'] = predictions.argmax(1)
#sub['prob'] = predictions.max(1)[0]
#sub.head()

In [ ]:
#sub[sub.prob>0.8].shape[0]/sub.shape[0]

In [ ]:
#sub.to_csv('submission.csv', index=False)


In [ ]:
# 0.8827